<a href="https://colab.research.google.com/github/davidnoone/PHYS332_FluidExamples/blob/main/Vortex_Dynamics_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vortex Dynamics in 2d

Created by Prof. David Noone (david.noone@auckland.ac.nz) 2021 (2025 update)


The behavior of vorticies in two dimensions (2d) can be captured by describing the unforced momentum equations. Consider an incompressible inviscid fluid that is near hydrostatic (i.e., like water, or air). With incompressibility, density is assumed constant, and mass continuity means that the divergence is zero.

$$
  \frac{\partial u}{dx} + \frac{\partial v}{dy} = 0
$$

This allows us to define a streamfunction as the only quantity needed to describe the velocity field. That is:

$$
    u = - \frac{\partial \psi}{\partial y}
$$

$$
    v = \frac{\partial \psi}{\partial x}
$$

The vorticity is found by taking the curl of the velocities, and is therefore related to the streamfunction as:

$$
  \Omega = \frac{\partial^2 \psi}{\partial x^2}
        + \frac{\partial^2 \psi}{\partial y^2}
$$

The evolution of vorticity can be predicted by taking a time derivative, leading to a remarkably elegant prognostic equation for vorticity:

$$
\frac{d \Omega}{d t} =
  \frac{\partial}{\partial x}(\frac{d v}{d t})
  - \frac{\partial}{\partial y}(\frac{d u}{d t}).
$$

We may note that the total derivative can be expanded to find the advective terms for each quantity "$A$", that is:

$$
 \frac{d A}{dt} = \frac{\partial A}{\partial t} +
    u \frac{\partial A}{\partial x} + v \frac{\partial A}{\partial y}.
$$

With the non-divergent condition, the advection term can be written using:

$$
  u \frac{\partial A}{\partial x} + v \frac{\partial A}{\partial y} =
  \frac{\partial (uA)}{\partial x} + \frac{\partial (vA)}{\partial y}   
$$

The final Eulerian prognostic model can therefore be described in a form that becomes particularly convenient for numerical methods:

$$
\frac{\partial \Omega}{\partial t} =
                      - \frac{\partial (u \Omega)}{\partial x}
                      - \frac{\partial (v \Omega)}{\partial y}
$$


## Solution scheme
From the above, it should be clear that the evolution of this system can be developed as follows:

  1. Beginning with $\Omega^n$ at some initial time ($\Omega$ could be calculated if $u$ and $v$ are known, of course).
  2. Solve the Poisson equation to evaluate the streamfunction from the known vorticity. i.e., obtain $\psi^n$.
  3. Calculate the velocities $(u,n)^n$ as the gradients in $\psi^n$.
  4. Use the velocities to calculate the advection of $\Omega$.
  5. Finally, advance $\Omega$ in time: step from time $n$ to $n+1$

This scheme can be repeated until the desired integration in time is complete.


# Solution method

Finite difference methods can be used to evaluate gradients. The Poisson equation is solved using an iterative method that applies the finite difference analog of the Laplacian in 2d.  If advection is evaluated using finite difference methods, a substantial amount of diffusion is needed to stabilize the numerical result. This is possible, but a more accurate scheme can also be used with better results. The stepping in time is done using a 3rd order scheme, rather than a simple single forward step, because it has some nice stability properties. This is accomplished using the weighted result from three applications of a simple forward Euler time step.





In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from numba import njit, prange

import time

print("Modules imported.")

Some run time initialization

In [ ]:
# Domain dimensions
nx = 50          # must be even
ny = 50          # even or odd

dx = 1.0        # grid spacing [metres]
dtime = 0.5     # time step [seconds]

nelapse = 100    # number of steps to run
nskip = 1        # steps between plotting output
nstep = 0

xmax = nx*dx
ymax = ny*dx
xmid = np.linspace(0,xmax,nx)
ymid = np.linspace(0,ymax,ny)

# Boundary conditions: top and bottom streamfunction
psi_top = 0.0
psi_bot = 0.0

vmag  = 1.0         # magnitude, for point vorticies
vpert = 0.01         # random initial perturbation

kdiff = 0.0*dx*dx/50.            # [Optional] diffusivity m2/sec (little bit for stability)

# Define the working arrays here to make them global (mostly so we can plot them)
vort = np.zeros((ny,nx)) + vpert*np.random.randn(ny,nx)
psi  = np.zeros((ny,nx))
uwnd = np.zeros((ny,nx))
vwnd = np.zeros((ny,nx))

# Initialize some structure: stick some vorticies various places

def add_vortex(xpos, ypos, width):
  xx,yy = np.meshgrid(xmid,ymid)
  dist = np.sqrt((xx-xpos)**2 + (yy-ypos)**2)/width
  vone = np.exp(-dist**2)     # gaussian
#  dist[dist > 1.0] = 0
#  vone = (1.0-dist)**2       # parabolic
  return vone

width = 4.
vort  =  vort - vmag*add_vortex(1*xmax/3, 1*ymax/3, width)    #  *** PLACE ONE VORTEX ***
#vort  =  vort + vmag*add_vortex(2*xmax/3, 1*ymax/3, width)
#vort  =  vort + vmag*add_vortex(1*xmax/3, 2*ymax/3, width)
#vort  =  vort + vmag*add_vortex(2*xmax/3, 2*ymax/3, width)

# Make sure it has zero mean
vort = vort - np.mean(vort)

print('Initialization complete.')

##Functions for simple finite difference operations.

A collection of finite difference operators are needed later on. Use the numpy "roll" function to implement periodic boundary conditions, but explictly use one-sided derivatives in the "$y$" direction at boundary edges.

In [ ]:
# Centered finite difference x derivative (2nd order), with periodic BCs: dF/dx
def fd_dfdx(fld,dx):
    dfdx = 0.5*(np.roll(fld,-1, axis=1) - np.roll(fld,+1, axis=1))/dx
    return dfdx

#Centered finite difference in y with single sided ends: dF/dy
def fd_dfdy(fld,dx):
#    dfdy = 0.5*(np.roll(fld,-1, axis=0) - np.roll(fld,+1, axis=0))/dx  # periodic
    dfdy = np.zeros_like(fld)
    dfdy[0     ,:] = (fld[1     ,:] - fld[     0,:])/dx
    dfdy[1:ny-1,:] = (fld[2:ny-0,:] - fld[0:ny-2,:])/(2*dx)
    dfdy[  ny-1,:] = (fld[  ny-1,:] - fld[  ny-2,:])/dx
    return dfdy

# Laplacian: del^2(F)
def fd_lap(fld,dx):
    flap =  -4*fld + np.roll(fld,-1, axis=1) + np.roll(fld,+1, axis=1)
    flap[0     ,:] = flap[0     ,:] + (fld[1     ,:]                )
    flap[1:ny-1,:] = flap[1:ny-1,:] + (fld[2:ny-0,:] + fld[0:ny-2,:])
    flap[  ny-1,:] = flap[  ny-1,:] + (                fld[  ny-2,:])
    return flap

# Curl/vorticity: del x (u,v) [Same as fd_div(v,-u)]
def fd_curl(u, v, dx):
    return fd_dfdx(v,dx) - fd_dfdy(u,dx)

# Divergence: del.(u,v)
def fd_div(u, v, dx):
    return fd_dfdx(u,dx) + fd_dfdy(v,dx)

# Advection opperator using finite differences [not very accurate]
def advect(fld,u,v,dx):
    return -1.0*(u*fd_dfdx(fld,dx) + v*fd_dfdy(fld,dx))


## Advection using finite volume methods

A finite volume method is used to evaluate the (non-linear) advection. Finite volume methods have some appealing mass conservation properties, and they can be configured to help prevent noisy solutions that come about from low order approximations to the continuous equations. This is especially important for managing non-linear terms.

Here we use a second order scheme where the advection tendency is computed as

$$
\left( \frac{\partial \Omega}{\partial t}\right)_n =
  \frac{1}{\Delta x} \left( F_{i+1/2} - F_{i-1/2} \right).
$$

The fluxes $F$ are at the interfaces (locations at "half" levels), and these must be estimated by linear reconstruction. The details of the estimation gives rise to a robust and accurate scheme. Note that an estimate of the value at each interface can be obtained from either side. That is, the left side of the interface at $i-1/2$ can be estimated using values from the left or from the right side. Consider the two options:

$$
F_{L,i-1/2} = F_{i-1} + \frac{1}{2}\Delta F_{1-1/2}
$$

$$
F_{R,i-1/2} = F_{i} - \frac{1}{2}\Delta F_{1+1/2}.
$$

The half here emerges from wanting to extrapolate half a grid length, $\Delta x$, to get from the cell center to the cell interfaces. The gradients $\Delta F$ can be adjusted to ensure monotonicity (and piece wise linear), which equivalently ensures that no new oscillations are produced in the solution. The slope is set to be no larger (in magnitude) than estimates of $\Delta F$ from the upstream, downstream and centered pairs of values of $\Omega$ from the possible combinations in the range from $i-1$ to $i+1$. (_There are a range of specific limiting functions; the "minmod" method used here is simple to implement and works well._)


Finally, the flux is estimated as the mean of these two estimates. The mean takes a special form involving a correction associate with the maximum possible (wave) speed supported by the equations of motion. This correction term effectivivly stabilizes the solution.

$$
  F = \frac{1}{2} \left[ (F_L + F_R) + \alpha (\Omega_R - \Omega_L) \right]
$$

where $\alpha = max(u_L, u_R)$. More formally, $\alpha$ is the Jacobian of the flux $\partial F/\partial u$. If we were to select $\alpha = 0$, we would identically recover the centered finite difference method!

Using these fluxes for each interface the advection tendency is obtained.




In [ ]:
# Two-D advection using a 2nd order FV method
R = -1   # right
L = 1    # left

# Piecewise linear reconstruction of interface values, using a minmod slope limiter
def reconstruct(f, dx, axis):
    # Compute limited slope/gradient
    f_dx = 0.5*( np.roll(f,R,axis=axis) - np.roll(f,L,axis=axis) )   # centered difference
    f_dx = np.maximum(0., np.minimum(1., ( (f-np.roll(f,L,axis=axis)))/(f_dx + 1.0e-8*(f_dx==0)))) * f_dx
    f_dx = np.maximum(0., np.minimum(1., (-(f-np.roll(f,R,axis=axis)))/(f_dx + 1.0e-8*(f_dx==0)))) * f_dx

    # get left and right side estimates of the fluxes
    f_R = f + 0.5*f_dx
    f_L = f - 0.5*f_dx
    f_L = np.roll(f_L,R,axis=axis)
    return f_L, f_R

# Form the fluxes: Lax-Freidrichs type scheme
def advect_flux_1d(f, u, dx, axis):
    u_L, u_R = reconstruct(u, dx, axis)
    f_L, f_R = reconstruct(f, dx, axis)
    alpha = np.maximum( np.abs(u_L), np.abs(u_R) )
    flx = 0.5*((u_L*f_L + u_R*f_R) - alpha*(f_L - f_R))
    return flx

# Evaluate advective tendency as divergence of (interface) fluxes
def advect_tend(f,u,v,dx):
    dy = dx
    flx_x = advect_flux_1d(f, u, dx, 1)
    flx_y = advect_flux_1d(f, v, dy, 0)

    fadv = (np.roll(flx_x,L,axis=1) - flx_x)/dx + \
           (np.roll(flx_y,L,axis=0) - flx_y)/dy

    return fadv


## Inverting $\Omega$: the Poisson equation

Finding $\psi$ from $\Omega$ requires the solution of the Poisson equation

$$
  \nabla^2 \psi = \Omega.
$$

Accomplishing this is one of the main numerical tasks in this calculation. Solving this equation effectively solves for mass continuity by defining a streamfunction.

Using finite differences we have:

$$
  \frac{
        \left( \psi_{i+1,j} - 2\psi_{i,j} + \psi_{i-1,j} \right)
      }{(\Delta x)^2}
 + \frac{
      \left( \psi_{i,j+1} - 2\psi_{i,j} + \psi_{i,j-1} \right)
      }{(\Delta y)^2}  
  = \Omega_i.
$$

Let's assume $\Delta x = \Delta y$, and rearrange to obtain:

$$
  \psi_{i,j}  = \frac{1}{4} \left(
    \psi_{i-1,j} + \psi_{i+1,j} + \psi_{i,j-1} + \psi_{i,j+1}
    - \frac{\Omega_{i,j}}{(\Delta x)^2}
    \right)
$$

This result suggest we can compute an updated estimate of $\psi_{i,j}$ knowing the value of $\Omega_{i,j}$, and the values of $\psi$ nearby.

Notice the detail that for $i$ and $j$ as odd values, the right hand side has only even values ($i+1$, $i-1$, $j+1$, and $j-1$). One can imagine the white and black squares on a chess board. This equation can be solved for the black squares, needing only values from the white squares, then solved again for the white squares needing only values from the black squares. Indeed, the approach can be repeated to obtain increasingly better estimates of $\psi$. Usually numerous iterations are required to obtain the needed accuracy. This approach is called a relaxation method, and described as the Gauss-Seidel method.

The method can be accelerated by introducing an "over-relaxation" in which we take the view that any updated estimate is still likely to be an underestimate, so multiply the size of the update by some factor. That is,

$$
  \psi_{i,j}^*  = (1-w)\psi_{i,j} + \frac{w}{4} \left(
    \psi_{i-1,j} + \psi_{i+1,j} + \psi_{i,j-1} + \psi_{i,j+1}
    - \frac{\Omega_{i,j}}{(\Delta x)^2}
    \right)
$$

Where the * denotes the updated estimate and the parameter $w$ is tuned to accelerate the convergence in the range $1 \le w \lt 2$. The case of $w=1$ has no accelaration; larger numbers give greater acceleration, but can be unstable for noisy fields. Some care is needed in selecting the value of $w$.

Finally, the convergence is faster if we start with a good guess! One good guess is the value from the previous model timestep. While not implemented here, another good guess is from simple linear extrapolation in time,

$$
  \psi_{guess}^{n+1} = 2\psi^{n} - \psi^{n-1}.
$$



In [ ]:
# Poisson solver using finite differences

# initialize red/black indices
(jm, im) = np.meshgrid(np.arange(ny),np.arange(nx))
x = np.zeros((nx,ny),dtype=np.int32)
x[:,1:-1] = im[:,1:-1]+jm[:,1:-1]
ir, jr = np.where((x>0) & (x%2 == 0))
ib, jb = np.where((x>0) & (x%2 == 1))

# set periodic boundary conditions in the "i" direction
irp = ir + 1
irp[irp>nx-1] -= nx
irm = ir - 1
irm[irm<   0] += nx

ibp = ib + 1
ibp[ibp>nx-1] -= nx
ibm = ib - 1
ibm[ibm<   0] += nx

# Need same but for J for periodic case

# Solve possion equation using Red/Black Gauss-Siedel method
def psolve(psi_bot,psi_top,psi0,vor,dx):
    err_max = 1.0e-5      # acceptable fractional error.
    niter_max = 200       # maximum allowable iterations (usually less)
    w = 1.5               # Over relaxation parameter (1 to 2)
    dxsq = dx*dx

    # Set first guess and boundary conditions
    psi = psi0.copy()
    psi[ny-1,:] = psi_top
    psi[   0,:] = psi_bot

    # Iterative solver by relaxation: red then black
    psi_last = np.empty_like(psi)
    for niter in range(niter_max):
        psi_last[:,:] = psi
        psi[jr,ir] = (1-w)*psi[jr,ir] + 0.25*w*(psi[jr,irp] + psi[jr,irm] + psi[jr+1,ir] + psi[jr-1,ir] - vor[jr,ir]*dxsq)
        psi[jb,ib] = (1-w)*psi[jb,ib] + 0.25*w*(psi[jb,ibp] + psi[jb,ibm] + psi[jb+1,ib] + psi[jb-1,ib] - vor[jb,ib]*dxsq)

        err = np.sum(np.abs(psi - psi_last))/(np.sum(np.abs(psi)) + 1.0e-18)
        if err < err_max:
            break

    if (niter >= niter_max):
        print("PSOLVE: Failed to converge (good luck!) err=",err)

    return psi


## Main integration functions

The hard part is more or less done. We now just need to set up the main formation of the time derivatives ("tendencies"), following the numbered strategy given above. Then, finally, do the time stepping.



In [ ]:
def tendency():
    """ Evaluates the RHS for the vorticity equation:
         dz/dt = - V . del z + K del^2 z
    """
    global vort, uwnd, vwnd, psi, dx, psi_bot, psi_top
    global dtime, nstep, nskip
    psi = psolve(psi_bot,psi_top,psi.copy(),vort,dx)
    uwnd = -fd_dfdy(psi,dx)
    vwnd =  fd_dfdx(psi,dx)
    ztend = advect_tend(vort,uwnd,vwnd,dx) + kdiff*fd_lap(vort,dx)
    return ztend

def step_model(dtime):
    """ Performs one model timestep. F(n+1) = F(n) + dt*(dF/dt)
        Uses a 3rd order SSP RK scheme to integrate.
    """
    global vort
    vort0 = vort
    vort = (vort0 +             dtime*tendency() )
    vort = (vort0 +      vort + dtime*tendency() )/2.0
    vort = (vort0 + 2.0*(vort + dtime*tendency()))/3.0
    return


##Visualizing the output

We will want to visualize the model state, so we define a function to plot the vorticity, and velocity (streamfunction) field. We will use the animation feature to advance the model state until it is time for the next frame to be plotted.

In [ ]:
def init_frame():
    cf.set_array([])
    cl.set_array([])
    return cl,cf,

def advance_frame(w):
    global vort, uwnd, vwnd, psi
    global dtime, nstep, nskip
    global cl, cf

    cno = np.max(np.sqrt(uwnd*uwnd + vwnd*vwnd))*dtime/(1.41*dx)

    # INTEGRATE THE EQUATIONS IN TIME: update to the next save time
    for n in range(nskip):
        nstep = nstep + 1
        if (cno > 2):
            print('CFL instability detected: abort! cno=',cno)
        step_model(dtime)

    # Do the plot on the predefine plotting axes
    print('Frame: ',w,' nstep =',nstep, 'CFL=',cno)
    ax.set(title="Vorticity (color) and streamfunction (contours) nstep="+str(nstep).zfill(4))
    while ax.collections:
        ax.collections[-1].remove()

    cf = ax.contourf(xx,yy,vort)
    cl = ax.contour(xx,yy,psi,colors='black')

    return cl, cf,


In [ ]:
# Create an animation object, with frames created as the model runs
xx, yy = np.meshgrid(xmid,ymid)

fig = plt.figure(figsize=(6,6))
ax = plt.axes(xlim=(0, xmax), ylim=(0, ymax))
ax.set_aspect('equal')
ax.set(xlabel='X position [m]', ylabel='Y position [m]')
cf = ax.contourf(xx,yy,vort)
cl = ax.contour(xx,yy,psi,colors='black')

anim = animation.FuncAnimation(fig, advance_frame, init_func=init_frame, frames=nelapse,
                                          interval=20,repeat=True)
plt.close()



##Run the model!

To actually run the integration, all we need to do is initiate the animation function "anim".

In [ ]:
# Run the model by initiating the animation.
anim

#Experiments to Try

1. Place two vorticies with the same sign, but of the same size, within the domain. What happens? Does it matter how far apart they are?

2. Repeat the experiment, but make one vortex larger (say, 2x the size) than the other. Do you get the same result?

3. Redo experiment (1) but use 2 vorticies with opposite sign. What happens? Does it matter how far apart they are?

4. Initialize the field with random numbers! (One might consider this depicting random turbulence). Add a little bit of diffusion (look for "kdiff"). As time evolve, describe what happens to the average size of vorticies.


(Bonus) What about 4 vorticies? Pairs of symmetric or antisymmetric vorticies?


Note: Since the result here is shown as an animation, you need only include some still frames in your write report.